### README
- From the user favourites page in [つくおき](https://cookien.com/favorite/)
- Extract all recipes found inside
  - Title
  - Storage period
  - Ingredients
  - Instructions
---

In [1]:
OUTPUT_LEVEL = 1 # 1 - Ingredients only / 2 - All

### Init selenium stuff

In [2]:
import os.path

from dotenv import load_dotenv
load_dotenv()

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# Init chromedriver
chrome_options = Options()
chrome_options.add_argument("--headless") # turn off GUI
chrome_options.add_argument("--no-sandbox")

homedir = os.path.expanduser("~")
webdriver_service = Service(f"{homedir}/chromedriver/stable/chromedriver")

# Init browser
browser = webdriver.Chrome(service=webdriver_service, options=chrome_options)

---

### Open favorites page

In [4]:
browser.get("https://cookien.com/favorite/")

elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@id='wpmem_login']")))
print(browser.title)

print(elems[0].find_element(By.XPATH, "//div[@id='wpmem_login']").text)

| つくおき | 作り置き・簡単レシピサイト
ログイン
ユーザID または メールアドレス
パスワード
 ログイン状態を保存する  
パスワードをお忘れですか？ パスワードリセット
はじめての方はこちら 新規ユーザー登録


---
### Handle login

In [5]:
username = os.getenv("COOKIEN_USER")
password = os.getenv("COOKIEN_PASS")

In [6]:
# Login
login_username = elems[0].find_element(By.XPATH, "//input[@id='log']")
login_password = elems[0].find_element(By.XPATH, "//input[@id='pwd']")
login_submit = elems[0].find_element(By.XPATH, "//input[@type='submit'][@class='buttons']")

login_username.send_keys(username)
login_password.send_keys(password)

login_submit.click()

# Post-login / wait for page to load
print(browser.title)
elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='site-info clear']")))
print(elems[0].find_element(By.XPATH, "//div[@class='site-info clear']").text)

お気に入り一覧 | つくおき | 作り置き・簡単レシピサイト
© 2013-2023 つくおき All rights reserved.
当サイトの情報及び、使用している画像、テキスト等の無断転載を禁じます。


--- 

### Extract recipe data for each item found in favourites page

In [7]:
elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='articleList']")))
ingredients = []
for elem in elems[0].find_elements(By.XPATH, ".//a[@class='content_list_link']"):
    elem_text = elem.get_attribute("title")
    elem_url = elem.get_attribute("href")
    if OUTPUT_LEVEL >= 1: 
        print(f"# [{elem_text}]({elem_url})")

    # Open url in new tab
    original_window = browser.current_window_handle
    assert len(browser.window_handles) == 1
    browser.switch_to.new_window('tabaa')
    browser.get(elem_url)  

    # Extract elements
    sub_elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@id='content']")))

    # Storage period
    if OUTPUT_LEVEL > 1:
        print("##", sub_elems[0].find_element(By.XPATH, ".//div[@id='clock_reizouko']").text)
    
    # Ingredients
    if OUTPUT_LEVEL > 0:
        print("##", sub_elems[0].find_element(By.XPATH, ".//div[@id='r_contents']/h2").text.replace('\n', ', '))
        for ingredient in sub_elems[0].find_elements(By.XPATH, ".//div[@id='r_contents']/p"):
            print("\t", ingredient.text.replace('\n', ' '))
            ingredients.append(ingredient.text.replace('\n', ' '))

    # Instructions
    if OUTPUT_LEVEL > 1:
        print("##", sub_elems[0].find_element(By.XPATH, ".//div[@id='ins_contents']/h2").text.replace('\n', ', '))
        for instruction in sub_elems[0].find_elements(By.XPATH, ".//p[@class='ins_des']"):
            print("\t", instruction.text.replace('\n', ' '))

    # Close tab and go back to main window
    browser.close()
    browser.switch_to.window(original_window)
    print(50*"=")

# TODO: Automatically upload to Trello / Google sheets?

# [レモンペッパーチキン](https://cookien.com/recipe/43773/)
## 材料（保存容器大1個分）, 食べきりの場合 3～4人分
	 鶏もも肉 約400g
	 ◎油 大1
	 ◎レモン汁 大1/2
	 ◎中華スープの素（メモ1） 5g
	 ◎ブラックペッパー 1g
	 ◎チリパウダー 1g
# [なすとピーマンの肉みそ炒め](https://cookien.com/recipe/16285/)
## 材料（保存容器大1個分）, 食べきりの場合 3～4人分
	 豚ひき肉 約300g
	 ピーマン 4個
	 なす 2本
	 にんにく 1かけ
	 ◯塩 少々
	 ◎みりん 大2
	 ◎みそ 大1.5
	 ◎オイスターソース 大1
	 ごま油 大1（お好みで）
# [ミックスビーンズのトマトカレー煮込み](https://cookien.com/recipe/1557/)
## 材料（保存容器中1個分）, 食べきりの場合 3～4人分
	 ミックスビーンズ 1袋（約120g）
	 トマト 2個
	 玉ねぎ 1/2個（約150g）
	 顆粒コンソメ 小1
	 カレールー 約20g
# [ピーマンの塩こんぶおかか和え](https://cookien.com/recipe/3367/)
## 材料（保存容器中1個分）, 食べきりの場合 3～4人分
	 ピーマン（メモ1） 6個
	 ◎和風だしの素 小1/2
	 ◎塩こんぶ 2つまみ（約6g）
	 ◎おかか 小分け1袋（2.5g）
# [ほうれん草のチーズ和え](https://cookien.com/recipe/41127/)
## 材料（保存容器中1個分）, 食べきりの場合 3～4人分
	 ほうれん草 1袋（6～8株）
	 プロセスチーズ 約40g
	 ◎醤油 大1/2
	 ◎粉チーズ 小1
	 ◎砂糖 小1/2
	 ◎ブラックペッパー 少々
# [トマトときゅうりの塩こうじ和え](https://cookien.com/recipe/37579/)
## 材料（保存容器中1個分）, 食べきりの場合 3～4人分
	 トマト 1個（約150g）
	 きゅうり 1本
	 ◎塩こうじ（メモ1） 大1.5
	 ◎和風顆粒だし 小1/3
# [ほうれん草のピーナッツ和え](https://cookien.

In [8]:
browser.quit()

--- 

### Extract ingredients data and group same items

In [9]:
import re

parsed_items = {}
skipped_items = {}

for item in ingredients:
    item = re.sub(r'\（メモ\d+\）', '', item)
    item = item.replace('◎', '')
    item = item.replace('◯', '').strip()

    key, value = item.split(' ', 1) 
    key = key.strip()
    value = value.strip()
    
    if not any(ext in item for ext in ["油", "酢", "塩", "みりん", "みそ", "汁", "ソース", "ペッパー", "素", "コンソメ", "パウダー", "砂糖", "だし"]):
        if key in parsed_items:
            parsed_items[key] += f" {value}"
        else:
            parsed_items[key] = value    
    else:
        if key in skipped_items:
            skipped_items[key] += f" {value}"
        else:
            skipped_items[key] = value

result_list = [f"{key} {value}" for key, value in parsed_items.items()]

result_list

['鶏もも肉 約400g',
 '豚ひき肉 約300g',
 'ピーマン 4個 6個',
 'なす 2本 １本',
 'にんにく 1かけ',
 'ミックスビーンズ 1袋（約120g）',
 'トマト 2個 1個（約150g） 4個',
 '玉ねぎ 1/2個（約150g）',
 'カレールー 約20g',
 'おかか 小分け1袋（2.5g）',
 'ほうれん草 1袋（6～8株） 1袋（6～8株）',
 'プロセスチーズ 約40g',
 '粉チーズ 小1',
 'きゅうり 1本',
 '大豆水煮 1袋（固形量200g）',
 'ちりめんじゃこ 約30g',
 '水 200mL',
 'パプリカ（赤） １／２個',
 'パプリカ（黄） １／２個']

In [10]:
skipped_items

{'油': '大1',
 'レモン汁': '大1/2',
 '中華スープの素': '5g',
 'ブラックペッパー': '1g 少々',
 'チリパウダー': '1g',
 '塩': '少々',
 'みりん': '大2 大2',
 'みそ': '大1.5',
 'オイスターソース': '大1',
 'ごま油': '大1（お好みで）',
 '顆粒コンソメ': '小1',
 '和風だしの素': '小1/2',
 '塩こんぶ': '2つまみ（約6g）',
 '醤油': '大1/2 大1/2 小2 小1 大１／２',
 '砂糖': '小1/2 大1/2 大1',
 '塩こうじ': '大1.5',
 '和風顆粒だし': '小1/3',
 'ピーナッツ（うす塩）': '約50g',
 '白だし': '大2',
 '調味酢': '大１．５'}